In [3]:
import pandas as pd
import numpy as np
import os
import gzip
import time
import seaborn as sns
import matplotlib.pyplot as plt
import gdreg

%load_ext autoreload
%autoreload 2

In [1]:
DATA_PATH = "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot"
DIC_PANNOT_SEED = {    
    "proxy_0_100@s" : DATA_PATH + "/proxy_0_100/proxy_0_100@s.chr@.pannot_mat.npz",
    "proxy_100_1000@s" : DATA_PATH+"/proxy_100_1000/proxy_100_1000@s.chr@.pannot_mat.npz",
    "proxy_1000_10000@s" : DATA_PATH+"/proxy_1000_10000/proxy_1000_10000@s.chr@.pannot_mat.npz",
    "ldp5_proxy_10000@s" : DATA_PATH + "/ldp5_proxy_10000/ldp5_proxy_10000@s.chr@.pannot_mat.npz",
    "pannot_ldp5_proxy_0_100@s" : \
        DATA_PATH + "/pannot_ldp5_proxy_0_100/ldp5_proxy_0_100@s.chr@.pannot_mat.npz",
    "pannot_ldp5_proxy_100_1000@s" : \
        DATA_PATH + "/pannot_ldp5_proxy_100_1000/ldp5_proxy_100_1000@s.chr@.pannot_mat.npz",
    "pannot_ldp5_proxy_1000_10000@s" : \
        DATA_PATH + "/pannot_ldp5_proxy_1000_10000/ldp5_proxy_1000_10000@s.chr@.pannot_mat.npz",
    "exon@s" : DATA_PATH + "/exon/exon@s.chr@.pannot_mat.npz",
    "gene@s" : DATA_PATH + "/gene/gene@s.chr@.pannot_mat.npz",
    "exonic_gene@s" : DATA_PATH + "/exonic_gene/exonic_gene@s.chr@.pannot_mat.npz",
    "protein_domain@s" : DATA_PATH + "/protein_domain/protein_domain@s.chr@.pannot_mat.npz",
    "cS2G_all@s" : DATA_PATH + "/cS2G_all/cS2G_all@s.chr@.pannot_mat.npz",
    "cS2G_promoter@s" : DATA_PATH + "/cS2G_promoter/cS2G_promoter@s.chr@.pannot_mat.npz",
    "cS2G_other@s" : DATA_PATH + "/cS2G_other/cS2G_other@s.chr@.pannot_mat.npz",
    "Dixon_2015_H1_ESC@s" : DATA_PATH + "/Dixon_2015_H1_ESC/Dixon_2015_H1_ESC@s.chr@.pannot_mat.npz",
    "Dixon_2015_H1_MES@s" : DATA_PATH + "/Dixon_2015_H1_MES/Dixon_2015_H1_MES@s.chr@.pannot_mat.npz",
    "Dixon_2015_H1_MSC@s" : DATA_PATH + "/Dixon_2015_H1_MSC/Dixon_2015_H1_MSC@s.chr@.pannot_mat.npz",
    "Dixon_2015_H1_NPC@s" : DATA_PATH + "/Dixon_2015_H1_NPC/Dixon_2015_H1_NPC@s.chr@.pannot_mat.npz",
    "Dixon_2015_H1_TRO@s" : DATA_PATH + "/Dixon_2015_H1_TRO/Dixon_2015_H1_TRO@s.chr@.pannot_mat.npz",
}

DIC_PANNOT = {}
for term in DIC_PANNOT_SEED:
    for suffix in ["_common_common", "_common_lf", "_lf_lf"]:
        DIC_PANNOT[term.replace("@s", suffix)] = DIC_PANNOT_SEED[term].replace("@s", suffix)
for val in DIC_PANNOT.values():
    print(val)

/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_0_100/proxy_0_100_common_common.chr@.pannot_mat.npz
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_0_100/proxy_0_100_common_lf.chr@.pannot_mat.npz
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_0_100/proxy_0_100_lf_lf.chr@.pannot_mat.npz
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_100_1000/proxy_100_1000_common_common.chr@.pannot_mat.npz
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_100_1000/proxy_100_1000_common_lf.chr@.pannot_mat.npz
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_100_1000/proxy_100_1000_lf_lf.chr@.pannot_mat.npz
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_1000_10000/proxy_1000_10000_common_common.chr@.pannot_mat.npz
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_1000_10000/proxy_1000_10000_common_lf.chr@.pannot_mat.npz
/n/groups/price/martin/data_GDREG/UKBimp_337K_

In [4]:
# Unfinished files
for pannot in DIC_PANNOT:
    for CHR in range(1,23):
        if os.path.exists(DIC_PANNOT[pannot].replace("@", "%d" % CHR)) is False:
            print(pannot, CHR)

### Dixon_2015_shared

In [12]:
# dic_pannot = {
#     "Dixon_2015_H1_ESC@s" : DATA_PATH + "/Dixon_2015_H1_ESC/Dixon_2015_H1_ESC@s.chr@.pannot_mat.npz",
#     "Dixon_2015_H1_MES@s" : DATA_PATH + "/Dixon_2015_H1_MES/Dixon_2015_H1_MES@s.chr@.pannot_mat.npz",
#     "Dixon_2015_H1_MSC@s" : DATA_PATH + "/Dixon_2015_H1_MSC/Dixon_2015_H1_MSC@s.chr@.pannot_mat.npz",
#     "Dixon_2015_H1_NPC@s" : DATA_PATH + "/Dixon_2015_H1_NPC/Dixon_2015_H1_NPC@s.chr@.pannot_mat.npz",
#     "Dixon_2015_H1_TRO@s" : DATA_PATH + "/Dixon_2015_H1_TRO/Dixon_2015_H1_TRO@s.chr@.pannot_mat.npz",
# }
# out_file = DATA_PATH + "/Dixon_2015_shared/Dixon_2015_shared@s.chr@.pannot_mat.npz"

# for CHR in [22]:
#     for mbin in ['_common_common', '_common_lf', '_lf_lf']:
#         mat_G = None
#         for pannot in dic_pannot:
#             temp_mat_G = gdreg.util.read_pannot_mat(
#                 dic_pannot[pannot].replace('@s', mbin).replace('@', '%d'%CHR)
#             )
#             print('CHR%d %s %s, size=%d' % (CHR, mbin, pannot, temp_mat_G.sum()))
            
#             if mat_G is None:
#                 mat_G = temp_mat_G
#             else:
#                 mat_G = mat_G * temp_mat_G
                
#             print('CHR%d %s %s, size=%d' % (CHR, mbin, 'Dixon_2015_shared', mat_G.sum()))
#         break
#     break
    

CHR22 _common_common Dixon_2015_H1_ESC@s, size=91066
CHR22 _common_common Dixon_2015_shared, size=91066
CHR22 _common_common Dixon_2015_H1_MES@s, size=87368
CHR22 _common_common Dixon_2015_shared, size=38104
CHR22 _common_common Dixon_2015_H1_MSC@s, size=71644
CHR22 _common_common Dixon_2015_shared, size=7552
CHR22 _common_common Dixon_2015_H1_NPC@s, size=197398
CHR22 _common_common Dixon_2015_shared, size=6734
CHR22 _common_common Dixon_2015_H1_TRO@s, size=137672
CHR22 _common_common Dixon_2015_shared, size=0


### Check correctness 

In [13]:
DATA_PATH

'/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot'

In [19]:
CHR = 22

# PGEN
PGEN_FILE = '/n/scratch3/users/j/jz286/imp_geno/ukb_imp_chr@_v3'
df_snp_chr = gdreg.util.read_pgen(PGEN_FILE.replace("@", "%d" % CHR))["pvar"]
df_snp_chr["MAF"] = gdreg.util.read_pgen(PGEN_FILE.replace("@", "%d" % CHR))["afreq"]["MAF"]
df_snp_chr["mbin"] = [gdreg.util.get_mbin(x) for x in df_snp_chr["MAF"]]
df_snp_chr.index = df_snp_chr['SNP']

# LD
mat_ld, dic_range = gdreg.util.read_ld(
    '/n/scratch3/users/j/jz286/imp_geno.gdreg_ld/ukb_imp_v3.c%d_s0_e10000_ld.npz' % CHR
)

# Gencode
df_gene = pd.read_csv("/n/groups/price/martin/data_GDREG/gene_annotation/ENSG_gene_annot_v41.txt", sep="\t")
df_gene_chr = df_gene.loc[df_gene['CHR'] == 'chr%d' % CHR]
df_exon = pd.read_csv("/n/groups/price/martin/data_GDREG/gene_annotation/ENSE_exon_annot_v41.txt", sep="\t")
df_exon_chr = df_exon.loc[df_exon['CHR'] == 'chr%d' % CHR]

file_folder = '/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/baseline_annot/vep'
df_pd = gdreg.util.read_annot(file_folder + '/ukb_imp_chr%s_v3.vep.annot.gz' % CHR)
dic_pd = {x:set(y.split(',')) for x,y in zip(df_pd['SNP'], df_pd['AN:DOMAINS'])}

# cS2G
df_cs2g = pd.read_csv(
        '/n/groups/price/martin/data_GDREG/gene_annotation/cS2G/cS2G_UKBB/cS2G.%s.SGscore.gz' % CHR, sep='\t',
    )
df_snpmap = pd.read_csv(
    '/n/groups/price/martin/data_GDREG/gene_annotation/cS2G/00_bim/UKBB.%s.info' % CHR, sep=' ',
)
temp_dic = {x:y for x,y in zip(df_snpmap['ID'], df_snpmap['RS'])}
df_cs2g['ID'] = df_cs2g['SNP']
df_cs2g['SNP'] = [temp_dic[x] for x in df_cs2g['ID']]
df_cs2g.index = df_cs2g['SNP']

# Loop
temp_path = '/n/groups/price/martin/data_GDREG/gene_annotation/3dgenome_loops'
temp_dic = {
    "Dixon_2015_H1_ESC" : temp_path + '/Dixon_2015.H1-ESC.hg19.peakachu-merged.loops',
    "Dixon_2015_H1_MES" : temp_path + '/Dixon_2015.H1-MES.hg19.peakachu-merged.loops',
    "Dixon_2015_H1_MSC" : temp_path + '/Dixon_2015.H1-MSC.hg19.peakachu-merged.loops',
    "Dixon_2015_H1_NPC" : temp_path + '/Dixon_2015.H1-NPC.hg19.peakachu-merged.loops',
    "Dixon_2015_H1_TRO" : temp_path + '/Dixon_2015.H1-TRO.hg19.peakachu-merged.loops',
}
dic_loop = {}
for term in temp_dic:
    dic_loop[term] = pd.read_csv(temp_dic[term], sep='\t', header=None)
    dic_loop[term].columns = ['CHR1', 'START1', 'END1', 'CHR2', 'START2', 'END2']
    dic_loop[term] = dic_loop[term].loc[dic_loop[term]['CHR1']==dic_loop[term]['CHR2']].copy()
    dic_loop[term].sort_values(['CHR1', 'START1', 'START2'], inplace=True)
    dic_loop[term] = dic_loop[term].loc[(dic_loop[term]["CHR1"] == 'chr%d'%CHR)]

In [52]:
for pAN in DIC_PANNOT:
# for pAN in ['Dixon_2015_H1_ESC_common_common']:
    print(pAN)
    print(DIC_PANNOT[pAN])
    mat_G = gdreg.util.read_pannot_mat(DIC_PANNOT[pAN].replace("@", "%d" % CHR))
    print('    mat_G', mat_G.shape, mat_G.getformat())
    
    n_print = 0
    i=0
    while i < 20000:      
        if mat_G.indptr[i]==mat_G.indptr[i+1]:
            i += 1
            continue    
            
        j = mat_G.indices[mat_G.indptr[i]:mat_G.indptr[i+1]][0] 
        
        print('    ------------------------------------------------------------------------------')
        print('    MAF = %0.4f, %0.4f,    dist = %d,    LD=%0.4f' % (
            df_snp_chr['MAF'][i], df_snp_chr['MAF'][j], df_snp_chr['BP'][i] - df_snp_chr['BP'][j], 
            mat_ld[i,j] if i<10000 else np.nan
        ))
        
        # Gene & Exon & Domain
        if pAN.startswith(('exon', 'gene', 'protein')):
            for idx in [i,j]:
                snp,bp = df_snp_chr['SNP'][idx],df_snp_chr['BP'][idx]
                ind_select = (df_gene_chr['START']<=bp) & (df_gene_chr['END']>=bp)
                gene = ','.join(df_gene_chr.loc[ind_select, 'GENE_NAME'])
                ind_select = (df_exon_chr['START']<=bp) & (df_exon_chr['END']>=bp)
                exon = ','.join(df_exon_chr.loc[ind_select, 'ENSE'])
                domain = dic_pd[snp] if snp in dic_pd else ''
                print('    %s' % snp)
                print('        gene = %s' % gene)
                print('        exon = %s' % exon)
                print('        domain = %s' % domain)
        
        # cS2G
        if pAN.startswith('cS2G'):
            for idx in [i,j]:
                snp,bp = df_snp_chr['SNP'][idx],df_snp_chr['BP'][idx]
                if snp in df_cs2g.index:
                    display(df_cs2g.loc[[snp], ['SNP', 'GENE', 'cS2G', 'INFO']])
            
        # Loops
        if pAN.startswith('Dixon_2015'):
            bp1 = min(df_snp_chr['BP'][i], df_snp_chr['BP'][j])
            bp2 = max(df_snp_chr['BP'][i], df_snp_chr['BP'][j])
            for term in dic_loop:
                ind_select1 = (dic_loop[term]['START1']<=bp1) & (dic_loop[term]['END1']>=bp1)
                ind_select2 = (dic_loop[term]['START2']<=bp2) & (dic_loop[term]['END2']>=bp2)
                ind_select = ind_select1 & ind_select2
                if ind_select.sum()==0:
                    continue
                else:
                    print('    %s n_loop=%d' % (term, ind_select.sum()))
                    
        i+=10
        n_print+=1
        if n_print>2:
            break
            
    
    print('==================================================================================')
#     break

proxy_0_100_common_common
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_0_100/proxy_0_100_common_common.chr@.pannot_mat.npz
    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.0809, 0.0800,    dist = -67,    LD=0.9956
    ------------------------------------------------------------------------------
    MAF = 0.0543, 0.0543,    dist = -81,    LD=0.9990
    ------------------------------------------------------------------------------
    MAF = 0.0530, 0.0538,    dist = -77,    LD=0.9343
proxy_0_100_common_lf
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/proxy_0_100/proxy_0_100_common_lf.chr@.pannot_mat.npz
    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.0221, 0.3130,    dist = -1,    LD=-0.0740
    ------------------------------------------------------------------------------
    MAF = 0.3139, 0.0438,  

    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.0315, 0.0315,    dist = -5,    LD=1.0086
    ------------------------------------------------------------------------------
    MAF = 0.0360, 0.0364,    dist = -38,    LD=1.0250
    ------------------------------------------------------------------------------
    MAF = 0.0320, 0.0317,    dist = -1,    LD=1.0250
pannot_ldp5_proxy_100_1000_common_common
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/pannot_ldp5_proxy_100_1000/ldp5_proxy_100_1000_common_common.chr@.pannot_mat.npz
    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.1020, 0.1051,    dist = -592,    LD=0.9836
    ------------------------------------------------------------------------------
    MAF = 0.3077, 0.3130,    dist = -288,    LD=0.9513
    -------------------------------------------------------------------

    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.0740, 0.0740,    dist = -583,    LD=0.9993
    rs2236639
        gene = CCT8L2
        exon = ENSE00001806026
        domain = {'Superfamily:SSF52029', 'Pfam:PF00118', 'PANTHER:PTHR11353', 'PANTHER:PTHR11353:SF70', 'Gene3D:3.50.7.10'}
    rs5747988
        gene = CCT8L2
        exon = ENSE00001806026
        domain = {'Low_complexity_(Seg):seg', 'Pfam:PF00118', 'Superfamily:SSF48592', 'PANTHER:PTHR11353', 'Gene3D:1.10.560.10', 'PANTHER:PTHR11353:SF70'}
    ------------------------------------------------------------------------------
    MAF = 0.3782, 0.3775,    dist = -438,    LD=0.9482
    rs112886011
        gene = TPTEP1
        exon = 
        domain = 
    rs2236640
        gene = TPTEP1
        exon = 
        domain = 
    ------------------------------------------------------------------------------
    MAF = 0.2021, 0.3782,    dist = 3462,    LD=0.6292

    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.0740, 0.0740,    dist = -583,    LD=0.9993
    rs2236639
        gene = CCT8L2
        exon = ENSE00001806026
        domain = {'Superfamily:SSF52029', 'Pfam:PF00118', 'PANTHER:PTHR11353', 'PANTHER:PTHR11353:SF70', 'Gene3D:3.50.7.10'}
    rs5747988
        gene = CCT8L2
        exon = ENSE00001806026
        domain = {'Low_complexity_(Seg):seg', 'Pfam:PF00118', 'Superfamily:SSF48592', 'PANTHER:PTHR11353', 'Gene3D:1.10.560.10', 'PANTHER:PTHR11353:SF70'}
    ------------------------------------------------------------------------------
    MAF = 0.1397, 0.1387,    dist = -559,    LD=0.9931
    rs5748622
        gene = XKR3
        exon = ENSE00001305313,ENSE00003916355
        domain = {'PANTHER:PTHR14297:SF7', 'PANTHER:PTHR14297'}
    rs5748623
        gene = XKR3
        exon = ENSE00001305313,ENSE00003916355
        domain = {'Pfam:PF09815', 'PANTHER:PTHR14297:

,SNP,GENE,cS2G,INFO
SNP,,,,
rs4008588,rs4008588,TPTEP1,1.0,|GTeX_Finemapped=1


...

    ------------------------------------------------------------------------------
    MAF = 0.0482, 0.0300,    dist = -481714,    LD=0.0206


,SNP,GENE,cS2G,INFO
SNP,,,,
rs370558029,rs370558029,CECR7,1.0,|ABC=1


,SNP,GENE,cS2G,INFO
SNP,,,,
rs532305349,rs532305349,CECR7,1.0,|ABC=1


Dixon_2015_H1_ESC_common_common
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/Dixon_2015_H1_ESC/Dixon_2015_H1_ESC_common_common.chr@.pannot_mat.npz
    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.1876, 0.3486,    dist = -159894,    LD=nan
    Dixon_2015_H1_ESC n_loop=1
    ------------------------------------------------------------------------------
    MAF = 0.4747, 0.3486,    dist = -157491,    LD=nan
    Dixon_2015_H1_ESC n_loop=1
    ------------------------------------------------------------------------------
    MAF = 0.1756, 0.3486,    dist = -155171,    LD=nan
    Dixon_2015_H1_ESC n_loop=1
Dixon_2015_H1_ESC_common_lf
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/Dixon_2015_H1_ESC/Dixon_2015_H1_ESC_common_lf.chr@.pannot_mat.npz
    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.1876, 0.0293,    dis

    ------------------------------------------------------------------------------
    MAF = 0.0458, 0.2958,    dist = -356404,    LD=-0.0092
    Dixon_2015_H1_NPC n_loop=1
    ------------------------------------------------------------------------------
    MAF = 0.2178, 0.0237,    dist = -354365,    LD=0.0020
    Dixon_2015_H1_NPC n_loop=1
Dixon_2015_H1_NPC_lf_lf
/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/Dixon_2015_H1_NPC/Dixon_2015_H1_NPC_lf_lf.chr@.pannot_mat.npz
    mat_G (199680, 199680) csr
    ------------------------------------------------------------------------------
    MAF = 0.0458, 0.0237,    dist = -355958,    LD=-0.0009
    Dixon_2015_H1_NPC n_loop=1
    ------------------------------------------------------------------------------
    MAF = 0.0465, 0.0237,    dist = -352450,    LD=0.0002
    Dixon_2015_H1_NPC n_loop=1
    ------------------------------------------------------------------------------
    MAF = 0.0486, 0.0237,    dist = -350261,    LD

### Old code